# 0. Install Dependencies

**Environment and Libs Setup**  

* OS libs:  
sudo apt install swig cmake libopenmpi-dev zlib1g-dev   
  
  
* Conda environment:  
conda create -n tf23_rl python=3.8 ipykernel jupyter pip   
conda activate tf23_rl  
  
  
* Python Libraries:  
pip install tensorflow==2.3 keras keras-rl2 box2d box2d-kengz atari-py  
python -m atari_py.import_roms \<path_to_ROMs\>  


* Including env to Jupyter notebook:  
ipython kernel install --user --name=TF2.3_RL_Gym  

[Referência para atari-py](https://github.com/openai/atari-py#roms)

In [24]:
#!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym 
import random

In [3]:
from gym import envs
print(envs.registry.all())

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v3), EnvSpec(BipedalWalkerHardcore-v3), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v0), EnvSpec(FrozenLake8x8-v0), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v3), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(HalfCheetah-v3), EnvSpec(Hopper-v2), EnvSpec(Hopper-v3), EnvSpec(Swimmer-v2), EnvSp

In [4]:
import atari_py
atari_py.list_games()

['tetris', 'space_invaders']

In [5]:
env = gym.make('SpaceInvaders-v0')

SpaceInvaders-v0

Maximize your score in the Atari 2600 game SpaceInvaders.   
In this environment, the observation is an RGB image of the screen, which is an array of shape (210, 160, 3).  
Each action is repeatedly performed for a duration of kkk frames, where kkk is uniformly sampled from \{2, 3, 4\}.


In [6]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [7]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [8]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:155.0
Episode:2 Score:95.0
Episode:3 Score:110.0
Episode:4 Score:515.0
Episode:5 Score:110.0
Episode:6 Score:145.0
Episode:7 Score:210.0
Episode:8 Score:210.0
Episode:9 Score:135.0
Episode:10 Score:210.0


# 2. Create a Deep Learning Model with Keras

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

2021-09-29 12:37:44.149579: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-09-29 12:37:44.149601: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


# model.add(
# Convolution2D - Metodo para procurar diferenças nas imagens
#(32 - Numero de filtros para detectar diferenças nas imagens
#(8,8) - Tamanho da matriz em que o filtro será aplicado, 
# strides=(4,4) - Tamanho do passo em pixeis que a matriz filtro se desloca na imagem.
# activation='relu', 
# input_shape=(3,height, width, channels)))

In [11]:
model = build_model(height, width, channels, actions)

2021-09-29 12:37:44.983800: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-29 12:37:44.983831: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-29 12:37:44.983855: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (NBTITMA143765): /proc/driver/nvidia/version does not exist
2021-09-29 12:37:44.984136: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-29 12:37:45.011057: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 18000000

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# 3. Build Agent with Keras-RL

In [13]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [14]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [15]:
# Modelo foi removido como solução ao problema de treino. 
del model

In [16]:
model = build_model(height, width, channels, actions)

In [17]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3))

In [18]:
import time

#Tic
sys_start_time = time.perf_counter()

In [19]:
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

Training for 20000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
   925/20000: episode: 1, duration: 30.191s, episode steps: 925, steps per second:  31, episode reward: 250.000, mean reward:  0.270 [ 0.000, 30.000], mean action: 2.570 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  1425/20000: episode: 2, duration: 465.086s, episode steps: 500, steps per second:   1, episode reward: 105.000, mean reward:  0.210 [ 0.000, 30.000], mean action: 2.480 [0.000, 5.000],  loss: 295.787085, mean_q: 1.893052, mean_eps: 0.890875
  2643/20000: episode: 3, duration: 1361.943s, episode steps: 1218, steps per second:   1, episode reward: 300.000, mean reward:  0.246 [ 0.000, 30.000], mean action: 2.521 [0.000, 5.000],  loss: 0.674406, mean_q: 0.696700, mean_eps: 0.816985
  3289/20000: episode: 4, duration: 679.575s, episode steps: 646, steps per second:   1, episode reward: 80.000, mean reward:  0.124 [ 0.000, 20.0

In [20]:
#Toc
delta_t = time.perf_counter() - sys_start_time

# Exibindo o tempo utilizado para o processo.
print('\n \n')
if delta_t > 3600:
    delta_t = delta_t / 3600
    print("Elapsed time: %.1f hours" % ((delta_t)))
elif delta_t > 60 and delta_t <= 3600:
    delta_t = delta_t / 60
    print("Elapsed time: %.1f min" % ((delta_t)))
else:
    print("Elapsed time: %.1f s" % ((delta_t)))


 

Elapsed time: 5.8 hours


# 4. Running model

In [28]:
scores = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 290.000, steps: 819
Episode 2: reward: 290.000, steps: 936
Episode 3: reward: 135.000, steps: 635
Episode 4: reward: 395.000, steps: 1619
Episode 5: reward: 230.000, steps: 789
Episode 6: reward: 485.000, steps: 896
Episode 7: reward: 120.000, steps: 694
Episode 8: reward: 150.000, steps: 633
Episode 9: reward: 180.000, steps: 853
Episode 10: reward: 270.000, steps: 905


In [27]:
print("Mean score: %.1f +/- %.1f" % (np.mean(scores.history['episode_reward']),
                                     np.std(scores.history['episode_reward'])))

Mean score: 236.0 +/- 95.5


# 5. Saving trainned model

In [23]:
dqn.save_weights('models/SpaceInvaders_20000.h5f')

In [19]:
#del model, dqn

In [20]:
#model = build_model(height, width, channels, actions)

In [21]:
#dqn = build_agent(model, actions)
#dqn.compile(Adam(lr=1e-3))

In [22]:
#dqn.load_weights('SavedWeights/20k-Fast/dqn_weights.h5f')

In [23]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 165.000, steps: 776
Episode 2: reward: 120.000, steps: 788
Episode 3: reward: 155.000, steps: 827
Episode 4: reward: 160.000, steps: 843
Episode 5: reward: 155.000, steps: 786
Episode 6: reward: 80.000, steps: 422
Episode 7: reward: 155.000, steps: 785
Episode 8: reward: 205.000, steps: 912
Episode 9: reward: 105.000, steps: 686
Episode 10: reward: 135.000, steps: 797
143.5


In [30]:
env.close()